In [1]:
import shutil
from pathlib import Path

import bw2data as bwd
import bw2calc as bwc

This notebook creates minimalistic versions of the foreground, biosphere and impact assessment methods databases as lightweight data packages.

In [2]:
bwd.projects.set_current('hypercog')

In [3]:
assert 'biosphere3' in bwd.databases

minimal verison of the biosphere with only the nodes I need

In [4]:
if 'mini_biosphere' in bwd.databases:

    print(f"deleting {bwd.Database('mini_biosphere').name}")
    del bwd.databases['mini_biosphere']
    

deleting mini_biosphere


In [5]:
co2 = next(x for x in bwd.Database("biosphere3") 
           if x['name'] == 'Carbon dioxide, fossil'
           and x['categories'] == ('air',))

ch4 = next(x for x in bwd.Database("biosphere3") 
           if x['name'] == 'Methane, fossil'
           and x['categories'] == ('air',))

In [6]:
co2_dict = {k:v for k,v in co2.as_dict().items() if k not in ['id','code']}
ch4_dict = {k:v for k,v in ch4.as_dict().items() if k not in ['id','code']}

co2_dict['code'] = 'CO2'
ch4_dict['code'] = 'CH4'
co2_dict['database'] = 'mini_biosphere'
ch4_dict['database'] = 'mini_biosphere'

bio_db = bwd.Database("mini_biosphere")
bio_db.register()

co2_new = bio_db.new_activity(**co2_dict)
co2_new.save()

ch4_new = bio_db.new_activity(**ch4_dict)
ch4_new.save()

bio_db_dpgk = bio_db.datapackage()

Successfully switch activity dataset to database `mini_biosphere`
Successfully switch activity dataset to database `mini_biosphere`


minimal verion of impact assessment method

In [7]:
# map between ef in the original biosphere database and the simplified one
ef_keys_dict = {
co2.key:co2_new.key,
ch4.key:ch4_new.key
}

ipcc2013 = bwd.Method(('IPCC 2013', 'climate change', 'GWP 100a'))
required_cfs = [(ef_keys_dict.get(key),cf) for key,cf in ipcc2013.load() if key in ef_keys_dict]
ipcc_simplified = bwd.Method(('IPCC','simple'))
ipcc_simplified.write(required_cfs)
assert ipcc_simplified.filepath_processed().is_file()

In [8]:
ipcc_simplified.register()

In [9]:
ipcc_simplified.metadata

{'abbreviation': 'ipccs.9602a5c9eb31e1a9ba91f7805bfc9e98',
 'num_cfs': 2,
 'geocollections': ['world']}

example of foreground database

In [10]:
a_key = ("testdb", "machine_a")

act_a_def = {
    'name': 'a',
    'unit': 'kilogram',
    'exchanges': [{"input": co2_new.key, "type": "biosphere", "amount": 10},
                  {"input": a_key, "output":a_key,'type':'production','amount':1},
                  {"input": ch4_new.key, "type": "biosphere", "amount": 1},
                 ],
    }

db = bwd.Database("testdb")
db.write(
    {
    a_key : act_a_def,
    }
    )

Writing activities to SQLite3 database:


Not able to determine geocollections for all datasets. This database is not ready for regionalization.


0% [#] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 09/25/2022 21:22:50
  Finished: 09/25/2022 21:22:50
  Total time elapsed: 00:00:00
  CPU %: 0.00
  Memory %: 0.98


calculate via normal procedure

In [11]:
lca = bwc.LCA({db.get('machine_a'):1},method=ipcc_simplified.name)

In [12]:
lca.lci()
lca.lcia()
lca.score

39.70000076293945

In [13]:
db.process(csv=True)

In [14]:
bio_db.process(csv=True)

get the related datapackages

In [15]:
shutil.copyfile(src=ipcc_simplified.filepath_processed(),
                dst=Path.cwd()/ipcc_simplified.filename_processed());

In [16]:
shutil.copyfile(src=db.filepath_processed(),
                dst=Path.cwd()/db.filename_processed());

In [18]:
shutil.copyfile(
src=bio_db.filepath_processed(),
dst=Path.cwd()/bio_db.filename_processed());